# Formation RNAseq CEA - juin 2023

Session IFB : 5 CPU + 21 GB de RAM

# Part 6: Read Counts

   

- 0. 1 - About session for IFB core cluster
- 0. 2 - Parameters to be set or modified by the user
- 1 - Some checks as a precaution
- 2 - Gene level quantification using ``featureCounts``
- 3 - Monitoring disk usage

- This notebook contains heavy running cells in Section 2, so make sure before you launch these cells that they are adapted to your environment. <br>
For Samtools, the ``-m`` option is the **RAM memory size that will be used by each thread**! <br>
<blockquote>
        See options <code>--threads 3 -m 5G</code> in Samtools line, Section 2  <br>
        Adapt <code>-T 4</code> in featureCounts lines to set it to 70-80% of available CPU number. <br>
        Adapt <code>-s 0</code> in  line to fit your library preparation. In the current notebook version, this option is set to <code>0</code> as the librairies are unstranded. <br>
</blockquote>

<div class="alert alert-block alert-danger">
    <b>Values set in this notebook are valid for a 5-CPU session with access to 21 GB of RAM</b>. Ideally, use 70-80% of the CPU amount your system or session has. <br>
    DO NOT ask for more RAM than you can use.
</div>

---
## **Before going further**

<div class="alert alert-block alert-danger"><b>Caution:</b> 
Before starting the analysis, save a backup copy of this notebok : in the left-hand panel, right-click on this file and select "Duplicate"<br>
You can also make backups during the analysis. Don't forget to save your notebook regularly: <kbd>Ctrl</kbd> + <kbd>S</kbd> or click on the 💾 icon.
</div>

---

## 0.1 - About session for IFB core cluster

<em>loaded JupyterLab</em> : Version 3.2.1

In [3]:
## Code cell 1 ##

echo "=== Cell launched on $(date) ==="

echo "=== Current IFB session size: Medium (5CPU, 21 GB) ==="
jobid=$(squeue -hu $USER | awk '/jupyter/ {print $1}')
sacct --format=JobID,AllocCPUS,NODELIST -j ${jobid}

=== Cell launched on Wed May 31 18:50:00 CEST 2023 ===
=== Current IFB session size: Medium (5CPU, 21 GB) ===
       JobID  AllocCPUS        NodeList 
------------ ---------- --------------- 
33600830              5      cpu-node-7 
33600830.ba+          5      cpu-node-7 
33600830.0            5      cpu-node-7 


In [13]:
## Code cell 2 ##

module load samtools/1.10 subread/2.0.1

echo "===== bam sorting by names ====="
samtools --version | head -n 2
echo "===== gene level quantification ====="
featureCounts -v


===== bam sorting by names =====
samtools 1.10
Using htslib 1.10.2
===== gene level quantification =====

featureCounts v2.0.1



---

## 0.2 - Parameters to be set or modified by the user


- Using a full path with a `/` at the end, **define the folder** where you want or have to work with `gohome` variable:

In [14]:
## Code cell 3 ##

gohome="/shared/projects/2312_rnaseq_cea/"

echo "=== Home root folder is ==="
echo "${gohome}"
echo ""
echo "=== Working (personal) folder tree ==="
tree -d -L 2 "${gohome}$USER"
echo "=== current working directory ==="
echo "${PWD}"

=== Home root folder is ===
/shared/projects/2312_rnaseq_cea/

=== Working (personal) folder tree ===
/shared/projects/2312_rnaseq_cea/scaburet
├── Data
│   ├── fastq
│   └── sra
└── Results
    ├── fastp
    ├── fastqc
    ├── fastq_screen
    ├── featurecounts
    ├── logfiles
    ├── multiqc
    ├── qualimap
    ├── samtools
    └── star

13 directories
=== current working directory ===
/shared/ifbstor1/projects/2312_rnaseq_cea/pipeline


- With a `/` at the end, precise the folder where alignment ``_ALigned.SortedByCoord.out.bam`` files (produced by ``STAR``) are:

In [15]:
## Code cell 4 ##

#mappedfolder="${gohome}$USER/Results/star/"
mappedfolder="${gohome}allData/Results/star/"

- Please remember the path to uncompressed annotation file, especially if you changed it in previous notebooks.

In [16]:
## Code cell 5 ##

gtffile="${gohome}allData/Reference/extracted/genome_annotation.gtf"

---
## 1 - Initial checks

As we change session or even day, let's first check all files are there:

In [17]:
## Code cell 6 ##

mappedfolder=${mappedfolder}
echo "There are $(ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam | wc -l) bam files:"
ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam

There are 6 bam files:
/shared/projects/2312_rnaseq_cea/allData/Results/star/SRR12730408_Aligned.sortedByCoord.out.bam
/shared/projects/2312_rnaseq_cea/allData/Results/star/SRR12730409_Aligned.sortedByCoord.out.bam
/shared/projects/2312_rnaseq_cea/allData/Results/star/SRR12730410_Aligned.sortedByCoord.out.bam
/shared/projects/2312_rnaseq_cea/allData/Results/star/SRR12730411_Aligned.sortedByCoord.out.bam
/shared/projects/2312_rnaseq_cea/allData/Results/star/SRR12730412_Aligned.sortedByCoord.out.bam
/shared/projects/2312_rnaseq_cea/allData/Results/star/SRR12730413_Aligned.sortedByCoord.out.bam


## 2 - Gene level quantification using <code>featureCounts</code>

### **1- Tool presentation**

`featureCounts` is part of the package <a href="http://subread.sourceforge.net/">SubRead</a>, to be used with bash, and RSubRead, to be used with R. This tool allows to attribute mapped reads to their matching feature (exon, gene, promoter, ...) on the genome and summarize counts per feature.  
  
In SubRead user guide, developpers recommend to use <i>specialized transcript-level quantification tools [...] for counting reads to transcripts</i> (see section 6.2.5, page 34 of pdf manual you can download with this <a href="https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwie1YDqwpD_AhXgU6QEHVLnB14QFnoECAsQAQ&url=https%3A%2F%2Fbioconductor.org%2Fpackages%2Frelease%2Fbioc%2Fvignettes%2FRsubread%2Finst%2Fdoc%2FSubreadUsersGuide.pdf&usg=AOvVaw1b3PpVhTNokdJHARtYAXgf">link</a>). So we will only generate gene-level quantification with `featureCounts`.

In [7]:
## Code cell 7 ##

featureCounts -v


featureCounts v2.0.1



A simple usage to count paired-end sequencing data, then counts fragments instead or reads, is:  
<code>featureCounts -p -a annotation.gtf \ 
                  -o counts.txt \ 
                  alignment.bam</code>

The <code>feautureCounts</code> options correspond to: <br>
<blockquote>
    <code>-p</code>, to count fragments instead or reads (paired-end data) <br>
    <code>-a TEXT</code>, to locate annotation file, in GTF/GFF format by default (<code>-f</code> to be used to give other file format) that can be a <code>.gzip</code> one. <br>
    <code>-o TEXT</code>, to set filename for counts<br>
    then alignement files (it may be a list) come. Either in BAM or SAM format, they can be sorted by read names or by chromosomal coordinates.
</blockquote>

By default, multi-mapped reads (or fragments) are not considered unless we use `-M` option (see others parameters to set in manual, pdf pages 38 to 43).  
It will select <code>exon</code> lines in annotation file (<code>-T exon</code> by default) to attribute reads (or fragments). Then, it will count them according to <code>gene_id</code> meta-feature level (<code>-g gene_id</code> by default).

We will use some other options:  
<blockquote>
    <code>-s INTEGER</code>, to specify strandness. Possible values include: 0 (unstranded), 1 (stranded) and 2 (reversely stranded). Default value is 0. <br>
    <code>-T INTEGER</code>, to set the number of threads to use (default, 1) <br>
    <code>--verbose</code>, to get information for debugging, such as unmatched chromosome/contig names. <br>
    As temporary files are saved by default to directory specified in <code>-o</code> option, we won't use <code>--tmpDir STRING</code> option <br>
</blockquote>

For paired-end data, it is possible to ask for filtering fragments that have both ends aligned (`-B` option), on same chromose and strand (`-C`) and even separated by an insert distance (`-P`, included in `-d` and `-D` values).  

Besides, input files will be used as sorted by names. Even if `featureCounts` handles `.bam` files as fast as `samtools` does, we will nonetheless use `samtools` and `--donotsort` option. Indeed, some files may be bigger than supported by `featureCounts`.

### **2- Step preparation**

We need to create a destination folder...

In [8]:
## Code cell 8 ##

#featcountfolder="${gohome}$USER/Results/featurecounts/"
featcountfolder="${gohome}allData/Results/featurecounts/"
mkdir -p ${featcountfolder}
#tree -d -L 1 "${gohome}$USER/Results/"
tree -d -L 1 "${gohome}allData/Results/"

/shared/projects/2312_rnaseq_cea/allData/Results/
├── fastp
├── fastqc
├── fastq_screen
├── featurecounts
├── logfiles
├── multiqc
├── qualimap
├── samtools
└── star

9 directories


... and remember used folder for saved screen output files.

In [9]:
## Code cell 9 ##

logfolder="${gohome}allData/Results/logfiles/"
#logfolder="${gohome}$USER/Results/logfiles/"

In [10]:
## Code cell 10 ##

gtffile=${gtffile}
#gtffile="${gohome}allData/Reference/extracted/genome_annotation.gtf"

echo "Annotation file: ${gtffile}"

Annotation file: /shared/projects/2312_rnaseq_cea/allData/Reference/extracted/genome_annotation.gtf


### **3- Compute reads or fragments counts**

#### 3.1 - Running <code>featuresCounts</code> on individual samples

- Before you run the following cells, make sure that they are adapted to your environment. <br>
For Samtools, the ``-m`` option is the **RAM memory size that will be used by each thread**! <br>
<blockquote>
        See options <code>--threads 3 -m 5G</code> in Samtools line <br>
        Adapt <code>-T 4</code> in featureCounts lines to set it to 70-80% of available CPU number. <br>
        Adapt <code>-s 0</code> in  line to fit your library preparation. In the current notebook version, this option is set to <code>0</code> as the librairies are unstranded. <br>
</blockquote>

<div class="alert alert-block alert-danger">
    <b>Values set in this notebook are valid for a 5-CPU session with access to 21 GB of RAM</b>. Ideally, use 70-80% of the CPU amount your system or session has. <br>
    DO NOT ask for more RAM than you can use.
</div>

<div class="alert alert-block alert-warning">
    <b><code>-T</code> option in featureCounts command line doesn't have an impact during counting process</b>. It has an effect when BAM are sorted on the fly by features counts, but this is not the case here, as the input file is sorted before by samtools.
</div>

In [27]:
## Code cell 11 ##

logfile="${logfolder}featureCounts-gene-level-counts-samtoolsSort.log"
echo "Screen output is redirected to ${logfile}"

Screen output is redirected to /shared/projects/2312_rnaseq_cea/allData/Results/logfiles/featureCounts-gene-level-counts-samtoolsSort.log


In [ ]:
## Code cell 12 ##

# as time command does not redirect output
echo "operation starts at $(date)" >> ${logfile}

time for fn in $(ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam); do  
    
    mysortedbam=$(basename ${fn})
    id=${mysortedbam/_Aligned.sortedByCoord.out.bam/}
    echo "===== Processing sampleID: ${id}..." | tee -a ${logfile}
    
    # outputfiles
    mytempfile="${featcountfolder}${id}_Aligned.sortedByNames.bam"
    myoutfile="${featcountfolder}${id}_paired-unstranded"
    
    # bam sorting...
    echo "samtools starts at $(date)" >> ${logfile}
    samtools sort -n \
                --threads 2 -m 5G \
                --output-fmt BAM \
                -o ${mytempfile} \
                -T ${featcountfolder} \
                ${fn} \
                &>> ${logfile}
    echo "samtools ends at $(date)" >> ${logfile}

    # some user conversation to help being patient
    echo "...changing tool..." | tee -a ${logfile}

    # then featureCounts
    echo "featureCounts starts at $(date)" >> ${logfile}

    featureCounts -p -s 0 -T 4 \
                  -a "${gtffile}" \
                  -o "${myoutfile}.counts" \
                  ${mytempfile} \
                  --donotsort \
                  --verbose \
                  &>> ${logfile}
    echo "featureCounts ends at $(date)" >> ${logfile}
    
    # removing extra bam file... saving disk space
    rm ${mytempfile}
    
    echo "... done" | tee -a ${logfile}
    
done

echo "operation ends at $(date)" >> ${logfile}

echo "=== Files created after featureCounts ===" >> ${logfile}
ls -lh "${featcountfolder}" >> ${logfile}
echo "featureCounts generated $(ls "${featcountfolder}"*.counts | wc -l) count files." \
    | tee -a ${logfile}
echo "featureCounts generated $(ls "${featcountfolder}"*.counts.summary | wc -l) summary files." \
    | tee -a ${logfile}

===== Processing sampleID: SRR12730403...
...changing tool...
... done
===== Processing sampleID: SRR12730404...
...changing tool...
... done
===== Processing sampleID: SRR12730405...
...changing tool...
... done
===== Processing sampleID: SRR12730406...
...changing tool...
... done
===== Processing sampleID: SRR12730407...
...changing tool...
... done
===== Processing sampleID: SRR12730408...
...changing tool...
... done
===== Processing sampleID: SRR12730409...
...changing tool...
... done
===== Processing sampleID: SRR12730410...
...changing tool...
... done
===== Processing sampleID: SRR12730411...
...changing tool...
... done
===== Processing sampleID: SRR12730412...


#### 3.2 - Running <code>featuresCounts</code> on multiple samples

The above cell processed each sample file individually: <code>samtools</code> generated the sorted bam that was used immediately by <code>featureCounts</code>, and that was removed afterward.   
Therefore we obtain one count file per sample. Those count files could be later joined in a single count matrix for the next step of analysis.   

Alternatively, <code>featuresCounts</code> can handle multiple bam files at once, creating directly a single count matrix for all the samples.    
This requires that all the sorted bam files are available as input for <code>featuresCounts</code>.    
This was done beforehand for all 11 samples, by running the next 2 cells.   
Code cell 14 runs <code>samtools</code> to generate (and keep!) the sorted bam files.   
Code cell 16 runs <code>featuresCounts</code> only once, using the list of sorted bam filenames as input.  

To run those cells in your own project, simply change their type from *Markdown* to *Code*.


## (Code cell 13) ##

logfile="${logfolder}counts-gene-level-counts-samtoolsSort.log"
echo "Screen output is redirected to ${logfile}"

## (Code cell 14) ##

# as time command does not redirect output
echo "operation starts at $(date)" >> ${logfile}

time for fn in $(ls "${mappedfolder}"*_Aligned.sortedByCoord.out.bam); do  
    
    mysortedbam=$(basename ${fn})
    id=${mysortedbam/_Aligned.sortedByCoord.out.bam/}
    echo "===== Processing sampleID: ${id}..." | tee -a ${logfile}
    
    # outputfiles
    mytempfile="${featcountfolder}${id}_Aligned.sortedByNames.bam"

    
    # bam sorting...
    echo "samtools starts at $(date)" >> ${logfile}
    samtools sort -n \
                --threads 3 -m 5G \
                --output-fmt BAM \
                -o ${mytempfile} \
                -T ${featcountfolder} \
                ${fn} \
                &>> ${logfile}
    echo "samtools ends at $(date)" >> ${logfile}

done

## (Code cell 15) ##

logfile="${logfolder}counts-gene-level-counts-featureCounts.log"
echo "Screen output is redirected to ${logfile}"

## (Code cell 16) ##

# creating the list of input filenames
listSortedBam=$(ls "${featcountfolder}"*_Aligned.sortedByNames.bam)
echo "The list of input files is: ${listSortedBam}" | tee -a ${logfile}

myoutfile="${featcountfolder}allSample_paired-unstranded"

echo "The output file is: ${myoutfile}" | tee -a ${logfile}

# then running featureCounts
echo "featureCounts starts at $(date)" >> ${logfile}

featureCounts -p -s 0 -T 4 \
              -a "${gtffile}" \
              -o "${myoutfile}.counts" \
              ${listSortedBam} \
              --donotsort \
              --verbose \
              &>> ${logfile}
echo "featureCounts ends at $(date)" >> ${logfile}
       
echo "... done" | tee -a ${logfile}
echo "operation ends at $(date)" >> ${logfile}

echo "=== Files created after featureCounts ===" >> ${logfile}
ls -lh "${featcountfolder}" >> ${logfile}
echo "featureCounts generated $(ls "${featcountfolder}"*.counts | wc -l) count files." \
    | tee -a ${logfile}
echo "featureCounts generated $(ls "${featcountfolder}"*.counts.summary | wc -l) summary files." \
    | tee -a ${logfile}

We can see the proportions of alignements that match a single gene (if default `featureCounts` parameters were kept) thanks to logfile and a research for specifiy patterns (`grep -e PATTERN`).

In [49]:
## Code cell 17 ##

logfile="${logfolder}counts-gene-level-counts-featureCounts.log"

cat ${logfile} | grep -e "Successfully assigned alignments" -e "Process BAM"

|| Process BAM file SRR12730403_Aligned.sortedByNames.bam...                  ||
||    Successfully assigned alignments : 43209224 (78.0%)                     ||
|| Process BAM file SRR12730404_Aligned.sortedByNames.bam...                  ||
||    Successfully assigned alignments : 33717431 (77.2%)                     ||
|| Process BAM file SRR12730405_Aligned.sortedByNames.bam...                  ||
||    Successfully assigned alignments : 30674349 (77.7%)                     ||
|| Process BAM file SRR12730406_Aligned.sortedByNames.bam...                  ||
||    Successfully assigned alignments : 31942711 (77.6%)                     ||
|| Process BAM file SRR12730407_Aligned.sortedByNames.bam...                  ||
||    Successfully assigned alignments : 35514233 (79.1%)                     ||
|| Process BAM file SRR12730408_Aligned.sortedByNames.bam...                  ||
||    Successfully assigned alignments : 32357023 (77.6%)                     ||
|| Process BAM file SRR12730

In [50]:
## Code cell 18 ##

# to have explanations
echo "summary file"
cat "${myoutfile}.counts.summary"

summary file
Status	/shared/projects/2312_rnaseq_cea/allData/Results/featurecounts/SRR12730403_Aligned.sortedByNames.bam	/shared/projects/2312_rnaseq_cea/allData/Results/featurecounts/SRR12730404_Aligned.sortedByNames.bam	/shared/projects/2312_rnaseq_cea/allData/Results/featurecounts/SRR12730405_Aligned.sortedByNames.bam	/shared/projects/2312_rnaseq_cea/allData/Results/featurecounts/SRR12730406_Aligned.sortedByNames.bam	/shared/projects/2312_rnaseq_cea/allData/Results/featurecounts/SRR12730407_Aligned.sortedByNames.bam	/shared/projects/2312_rnaseq_cea/allData/Results/featurecounts/SRR12730408_Aligned.sortedByNames.bam	/shared/projects/2312_rnaseq_cea/allData/Results/featurecounts/SRR12730409_Aligned.sortedByNames.bam	/shared/projects/2312_rnaseq_cea/allData/Results/featurecounts/SRR12730410_Aligned.sortedByNames.bam	/shared/projects/2312_rnaseq_cea/allData/Results/featurecounts/SRR12730411_Aligned.sortedByNames.bam	/shared/projects/2312_rnaseq_cea/allData/Results/featurecounts/SRR12730

Let's have a look at the beginning of result files from featureCounts: 

In [18]:
## Code cell 19 ##

head -n 10 ${gohome}$USER/Results/featurecounts/*_paired-unstranded.counts

==> /shared/projects/2312_rnaseq_cea/scaburet/Results/featurecounts/SRR12730409_paired-unstranded.counts <==
# Program:featureCounts v2.0.1; Command:"featureCounts" "-p" "-s" "0" "-T" "4" "-a" "/shared/projects/2312_rnaseq_cea/allData/Reference/extracted/genome_annotation.gtf" "-o" "/shared/projects/2312_rnaseq_cea/allData/Results/featurecounts/SRR12730409_paired-unstranded.counts" "/shared/projects/2312_rnaseq_cea/allData/Results/featurecounts/SRR12730409_Aligned.sortedByNames.bam" "--donotsort" "--verbose" 
Geneid	Chr	Start	End	Strand	Length	/shared/projects/2312_rnaseq_cea/allData/Results/featurecounts/SRR12730409_Aligned.sortedByNames.bam
ENSMUSG00000102693.2	chr1	3143476	3144545	+	1070	0
ENSMUSG00000064842.3	chr1	3172239	3172348	+	110	0
ENSMUSG00000051951.6	chr1;chr1;chr1;chr1;chr1;chr1;chr1	3276124;3276746;3283662;3283832;3284705;3491925;3740775	3277540;3277540;3285855;3286567;3287191;3492124;3741721	-;-;-;-;-;-;-	6094	0
ENSMUSG00000102851.2	chr1	3322980	3323459	+	480	0
ENSMUSG00

As you can see, the genes are in rows and the counts in the samples are provided in the on-before-last column.   

## 4 - Monitoring disk usage

In [ ]:
## Code cell 20 ##

du -h -d2 ${gohome}$USER

We now have a personal folder that gets too heavy.  Let's remove some files we won't use anymore: 

- initial srr files in Data/sra/   
- raw fastq.gz files in Data/fastq/raw/   
- cleaned fastq.gz files in /Results/fastp   
- intermediate Aligned.sortedByNames.bam files produced by <code>samtools</code> fo <code>featuresCounts</code> just above

In [21]:
## Code cell 21 ##

# Saving disk space
# initial srr files
rm -r ${gohome}$USER/Data/sra/

# raw fastq.gz
rm ${gohome}$USER/Data/fastq/raw/*.fastq.gz

# cleaned fastq.gz
rm ${gohome}$USER/Results/fastp/*.fastp.fastq.gz

# intermediate Aligned.sortedByNames.bam
rm ${gohome}$USER/Results/featurecounts/*_Aligned.sortedByNames.bam

rm: cannot remove ‘/shared/projects/2312_rnaseq_cea/scaburet/Data/sra/’: No such file or directory
rm: cannot remove ‘/shared/projects/2312_rnaseq_cea/scaburet/Data/fastq/raw/*.fastq.gz’: No such file or directory
rm: cannot remove ‘/shared/projects/2312_rnaseq_cea/scaburet/Results/fastp/*.fastp.fastq.gz’: No such file or directory
rm: cannot remove ‘/shared/projects/2312_rnaseq_cea/scaburet/Results/featurecounts/*_Aligned.sortedByNames.bam’: No such file or directory


: 1

Let's see our disk usage now:

In [22]:
## Code cell 22 ##

du -h -d2 ${gohome}$USER

19M	/shared/projects/2312_rnaseq_cea/scaburet/Results/multiqc
70M	/shared/projects/2312_rnaseq_cea/scaburet/Results/featurecounts
1.8M	/shared/projects/2312_rnaseq_cea/scaburet/Results/fastp
1.1M	/shared/projects/2312_rnaseq_cea/scaburet/Results/samtools
252K	/shared/projects/2312_rnaseq_cea/scaburet/Results/logfiles
6.3M	/shared/projects/2312_rnaseq_cea/scaburet/Results/fastqc
12K	/shared/projects/2312_rnaseq_cea/scaburet/Results/star
4.0K	/shared/projects/2312_rnaseq_cea/scaburet/Results/.ipynb_checkpoints
20M	/shared/projects/2312_rnaseq_cea/scaburet/Results/fastq_screen
8.6M	/shared/projects/2312_rnaseq_cea/scaburet/Results/qualimap
125M	/shared/projects/2312_rnaseq_cea/scaburet/Results
12K	/shared/projects/2312_rnaseq_cea/scaburet/Data/fastq
12K	/shared/projects/2312_rnaseq_cea/scaburet/Data/.ipynb_checkpoints
40K	/shared/projects/2312_rnaseq_cea/scaburet/Data
8.0K	/shared/projects/2312_rnaseq_cea/scaburet/.ipynb_checkpoints
125M	/shared/projects/2312_rnaseq_cea/scaburet


---
___

Now we go on with an introduction to the R language, that will be used during the next steps of the analysis.  
  
**=> Step 7: Introduction to R** 

The jupyter notebook used for the next session will be *Pipe_07-R403_Intro-to-R.ipynb*    
Let's retrieve it in our personal directory, in order to have a private copy to work on:   

In [23]:
## Code cell 38 ##   

cp "${gohome}pipeline/Pipe_07-R403_Intro-to-R.ipynb" "${gohome}$USER/"
cp "${gohome}pipeline/Temperatures.txt" "${gohome}$USER/"

Bénédicte Noblet - 05-07 2021   
Sandrine Caburet - 02-05 2023   
Maj 24/05/2023